# Gallatin County parcel assessment parser

Code that takes Montana Cadastral data for Gallatin County and massages it into analysis-friendly forms.

Some of the workflow in producing/using the outputs here are in QGIS, also analysis in another notebook, gc-parcel-assessment-analyzer.

In [239]:
# Libraries
import pandas as pd
import numpy as np

import ast # used for ast.literal_eval function
import csv

In [240]:
# Read in & merge data

# Data comes from Montana Cadastral system
# Downloaded as an ESRI shapefile, then exported to a .csv using QGIS
# '2015' data downloaded in Sept. 2015, representing the 2014 assessment cycle
# '2014' data is for 2008 assessment cycle
# It came from an earlier (June 2015) download of cadastral data for Gallatin County,
# before the state had added in the 2014 numbers.

# Deep apologies to future self/readers for the opaque naming convention
# Also note — Nov. 2015 publication didn't end up focusing on property value change,
# so much of this analysis didn't end up being used at that point.

# Read in current parcel values & rename some columns
df2015 = pd.read_csv('gc-parcels-current-2015/gc-parcels-current-2015.csv', dtype={'parcelid':str,
                                                                                   'totalacres':float,
                                                                                   'totallandv':float,
                                                                                   'totalbuild':float,
                                                                                   'totalvalue':float})
df2015 = df2015.rename(columns={'totalacres': 'SizeAcres',
                                'totallandv': '2015LndVal',
                                'totalbuild': '2015BldVal',
                                'totalvalue': '2015TotVal',
                                'propertyid': '2015PropID'})
df2015 = df2015[df2015['parcelid'].notnull()] # removes rows w/ no geocode
df2015 = df2015[df2015['parcelid'].apply(len) == 17] # removes rows w/ non-standard-length geocode
df2015 = df2015.drop_duplicates(subset='parcelid') # removes rows w/ duplicate geocodes
                                                                               
# Read in data for 2008 assessment
df2014 = pd.read_csv('gc-parcels-june-2015/gc-parcels-june-2015.csv', dtype={'parcelid':str})
df2014 = df2014.rename(columns={'totalacres': '2014Acres',
                                'totallandv': '2014LndVal',
                                'totalbuild': '2014BldVal',
                                'totalvalue': '2014TotVal',
                                'propertyid': '2014PropID'})
df2014 = df2014[df2014['parcelid'].notnull()] # removes rows w/ no geocode
df2014 = df2014[df2014['parcelid'].apply(len) == 17] # removes rows w/ non-standard-length geocode
df2014 = df2014.drop_duplicates(subset='parcelid') # removes rows w/ duplicate geocodes

# Merge on 'parcelid' field
df = pd.merge(df2015, df2014, how='left', on='parcelid')

In [241]:
# Cleaning up owner names using pairings from external file
# e.g. 'US Forest Service' & 'Forest Service' --> 'UNITED STATES OF AMERICA''
def ownerNameClean(name):
    if name in ownerRename:
        return ownerRename[name]
    else:
        return name

with open('owner-name-corrections-reviewed.txt', 'r') as f:
    ownerRename = ast.literal_eval(f.read()) #iterprets file as dict
    
df['orig_ownName'] = df['ownername']
df['ownername'] = df['ownername'].apply(ownerNameClean)

In [242]:
# Parsing property types into categories

# Dictionary for converting property use designations to aggregated categories
# 'Raw data designation' : 'Sorted category'
propTypeDict = {'VR - Vacant Land Rural': 'Vacant',
                'VU - Vacant Land Urban': 'Vacant',
                'AR - Agricultural Rural': 'Agricultural',
                'AU - Agricultural Urban': 'Agricultural',
                'FU - Farmstead Urban' : 'Agricultural',
                'FR - Farmstead Rural' : 'Agricultural',
                'EP - Exempt Property': 'Tax-exempt',
                'CU - Commercial Urban': 'Commercial',
                'CR - Commercial Rural': 'Commercial',
                'RR - Residential Rural': 'Residential',
                'RU - Residential Urban': 'Residential',
                'TU - Townhouse Urban': 'Residential',
                'TR - Townhouse Rural': 'Residential',
                'BR - Commercial Condo Rural': 'Residential',
                'KU - Condominium Urban': 'Residential',
                'KR - Condominium Rural': 'Residential',
                'IU - Industrial Urban': 'Industrial',
                'IR - Industrial Rural': 'Industrial',
                'MU - Mixed Use / Urban': 'Mixed-use',
                'MR - Mixed Use / Rural': 'Mixed-use',
                'CN - Centrally Assessed Non-Valued Property': 'Other/No Data',
                'CA - Centrally Assessed': 'Other/No Data',
                'NV - Non-Valued Property': 'Other/No Data'   
               }

def propTypeSort(propType):
    if propType in propTypeDict:
        return propTypeDict[propType]
    else:
        # print "Can't parse: ", propType # For debugging
        return 'Other/No Data'
    
df['PropCat'] = df['proptype'].apply(propTypeSort)

In [243]:
# Adding an 'inside Bozeman?' column using external list of geocodes for properties within city limits
bznParcels = pd.read_csv('gc-parcels-inside-bzn.csv', dtype={'PARCELID':str,})

df['inbzn'] = np.where(df['parcelid'].isin(bznParcels['PARCELID']), True, False)

# Adding an 'inside NCOD?' columne using external list of geocodes
ncodParcels = pd.read_csv('gc-parcels-inside-ncod.csv', dtype={'PARCELID':str,})

df['inncod'] = np.where(df['parcelid'].isin(ncodParcels['PARCELID']), True, False)

In [244]:
# Calculating 2014 -> 2015 change in total value (only if both values are non-zero)
def calcChange(val14, val15):
    if val14 > 0 and val15 > 0:
        return val15 - val14
    else:
        pass
    
df['TotChange'] = df.apply(lambda row: calcChange(row['2014TotVal'], row['2015TotVal']), axis=1)
df['LndChange'] = df.apply(lambda row: calcChange(row['2014LndVal'], row['2015LndVal']), axis=1)
df['BldChange'] = df.apply(lambda row: calcChange(row['2014BldVal'], row['2015BldVal']), axis=1)

# Calculating percent change between 2014 and 2015
def perChange(change, val14):
    if (change >= 0 or change < 0) and val14 != 0:
        return change / val14
    else:
        pass

df['TotPerCh'] = df.apply(lambda row: perChange(row['TotChange'], row['2014TotVal']), axis=1)
df['LndPerCh'] = df.apply(lambda row: perChange(row['LndChange'], row['2014LndVal']), axis=1)
df['BldPerCh'] = df.apply(lambda row: perChange(row['BldChange'], row['2014BldVal']), axis=1)

# Calculating 2015 value per square foot
SF_PER_ACRE = 43560 # Conversion factor, number of square feet in an acre

def valPerSf(value, acres):
    if value > 0 and acres > 0:
        return value / (acres * SF_PER_ACRE)
    else:
        pass
    
df['TotValPSF'] = df.apply(lambda row: valPerSf(row['2015TotVal'], row['SizeAcres']), axis=1)
df['LndValPSF'] = df.apply(lambda row: valPerSf(row['2015LndVal'], row['SizeAcres']), axis=1)
df['BldValPSF'] = df.apply(lambda row: valPerSf(row['2015BldVal'], row['SizeAcres']), axis=1)

In [245]:
# Adding columns for how much total land each owner owns, total value and # of parcels
dfByOwner = df.groupby('ownername').agg({'ownername': 'count',
                                      'SizeAcres': 'sum',
                                      '2015TotVal': 'sum'})
dfByOwner = dfByOwner.rename(columns={'ownername': 'Number'})

def totValue(owner):
    try:
        return dfByOwner.loc[owner]['2015TotVal']
    except:
        pass
    
def totSize(owner):
    try:
        return dfByOwner.loc[owner]['SizeAcres']
    except:
        pass
    
def totParcels(owner):
    try:
        return dfByOwner.loc[owner]['Number']
    except:
        pass
    
df['ownerSize'] = df['ownername'].apply(totSize)
df['ownerVal'] = df['ownername'].apply(totValue)
df['ownerParcels'] = df['ownername'].apply(totParcels)

In [246]:
# Preview output
print df.columns
df.head(3)

Index([u'parcelid', u'2015PropID', u'assessment', u'subdivisio', u'addresslin', u'addressl_1', u'citystatez', u'propaccess', u'levydistri', u'proptype', u'SizeAcres', u'2015BldVal', u'2015LndVal', u'2015TotVal', u'ownername', u'owneraddre', u'owneradd_1', u'owneradd_2', u'ownercity', u'ownerstate', u'ownerzipco', u'careoftaxp', u'shape_area', u'2014PropID', u'2014Acres', u'2014BldVal', u'2014LndVal', u'2014TotVal', u'orig_ownName', u'PropCat', u'inbzn', u'inncod', u'TotChange', u'LndChange', u'BldChange', u'TotPerCh', u'LndPerCh', u'BldPerCh', u'TotValPSF', u'LndValPSF', u'BldValPSF', u'ownerSize', u'ownerVal', u'ownerParcels'], dtype='object')


,parcelid,2015PropID,assessment,subdivisio,addresslin,addressl_1,citystatez,propaccess,levydistri,proptype,...,BldChange,TotPerCh,LndPerCh,BldPerCh,TotValPSF,LndValPSF,BldValPSF,ownerSize,ownerVal,ownerParcels
0,06079817214070000,1528907,00RGG66579,WOODLAND PARK SUB PH 2,HATFIELD CT,NaN,"BOZEMAN, MT 59715",NaN,06-235041-7R 41,VR - Vacant Land Rural,...,NaN,NaN,NaN,NaN,8.579316,8.579316,NaN,53.536,1118514,15
1,06079817214110000,1528909,00RGG66581,WOODLAND PARK SUB PH 2,139 GRANITE PEAK DR,NaN,"BOZEMAN, MT 59715",NaN,06-235041-7R 41,VR - Vacant Land Rural,...,NaN,NaN,NaN,NaN,8.692427,8.692427,NaN,1.200,443075,7
2,06079817214010000,1528904,00RGG66576,WOODLAND PARK SUB PH 2,HATFIELD CT,NaN,"BOZEMAN, MT 59715",NaN,06-235041-7R 41,VR - Vacant Land Rural,...,NaN,NaN,NaN,NaN,8.848875,8.848875,NaN,0.186,71695,1


In [252]:
# Outputting select portions of data for mapping via import/merging with shapefile in QGIS
OUTPUT_FILE_NAME = 'gc-parcels-for-gis'

output_columns = ['parcelid', 'PropCat', 'addresslin',
                  'SizeAcres', 'inbzn', 'inncod', 'ownername',
                  '2015BldVal', '2015LndVal', '2015TotVal',
                  'TotChange', 'LndChange', 'BldChange',
                  'TotValPSF', 'LndValPSF', 'BldValPSF',
                  'ownerSize', 'ownerVal', 'ownerParcels']

dfOut = df[output_columns]
dfOut.to_csv(OUTPUT_FILE_NAME + '.csv') 

# Create .csvt so QGIS doesn't convert PARCELID to a numeric value and #$#% up the join
# See: http://anitagraser.com/2011/03/07/how-to-specify-data-types-of-csv-columns-for-use-in-qgis/

csvt = '"Integer",' # entry for index column

for column in dfOut.columns:
    if column == 'PARCELID':
        csvt += '"String(17)",'
    elif dfOut[column].dtype == 'object':
        csvt += '"String",'
    elif dfOut[column].dtype == 'bool':
        csvt += '"String",' # no bool option in csvt spec
    elif dfOut[column].dtype == 'float64':
        csvt += '"Real",'
    else:
        csvt += '"",' # Not sure if this works w/ .csvt spec

csvt = csvt[:-1] # removes trailing comma
        
with open(OUTPUT_FILE_NAME + '.csvt', 'w') as f:
    f.write(csvt)

In [248]:
# Outputting complete dataframe for analysis in another notebook
df.to_csv('gc-parcels-complete.csv')

In [249]:
# List of owner names for checking standardization process
df['ownername'].drop_duplicates().to_csv('gc-owner-list.csv')

In [250]:
# Export to JSON for treemap viz via d3js
# Abandoned approach — turns out my web browser has a hard time animating 40K <div>s simultaneously

# propCats = df['PropCat'].unique()

# for category in propCats:
#     dfTreemap = df[df['PropCat'] == category].groupby('ownername').agg({'ownername': 'count',
#                                           'SizeAcres': 'sum',
#                                           '2015TotVal': 'sum'})
#     dfTreemap = dfTreemap.rename(columns={'ownername': 'parcels',
#                                           'SizeAcres': 'totacres',
#                                           '2015TotVal': 'totval'
#                                        })
#     dfTreemap.reset_index(level=0, inplace=True)
#     # separate b/c parcels column is also initally labeled 'ownername'
#     dfTreemap = dfTreemap.rename(columns={'ownername': 'name'})

#     # Hacky way to catch '/' in other category messing up filepath
#     if category == 'Other/No Data':
#         category = 'Other or No Data'
    
#     print '\n' + category
#     print dfTreemap.head(2)
    
#     filepath = 'gc-parcel-treemap-viz/' + category + '-output.json'
#     dfTreemap.to_json(filepath,orient='records')